In [33]:
# Import libraries
import boto3
import json
import heapq
from collections import deque
# import itertools
# import pandas as pd
# from io import StringIO

In [2]:
# Set variables for s3 connection
region = 'ap-northeast-2'
bucket = 'emr-data-sync'
prefix = 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/'
max_keys = 1000
delimiter = '/'

In [3]:
# S3 onnection with boto3
s3 = boto3.client('s3', region_name=region)
response = s3.list_objects(Bucket=bucket)

objects = s3.list_objects_v2(Bucket=bucket,
                             Prefix=prefix,
                             MaxKeys=max_keys,
                             Delimiter=delimiter)

print(f"Folders Count : {objects['KeyCount']}")
print()
print(f"If \"Folders Count\" is near 1,000 Please Examine \"max_keys\"")
print(f">>> max_keys = {max_keys}")

Folders Count : 145

If "Folders Count" is near 1,000 Please Examine "max_keys"
>>> max_keys = 1000


In [7]:
# Nested folders list to dequeue
folder_prefix_queue = deque()

for obj in objects.get('CommonPrefixes'):
    folder_prefix_queue.appendleft(obj.get('Prefix'))

print(f"Dequeued Folders Count : {len(folder_prefix_queue)}\n")
if objects['KeyCount'] == len(folder_prefix_queue):
    print(f"All Folders Successfully Dequeued")
else:
    print(f"Failed to Deque All Folders.\nPlease Check The Code")

Dequeued Folders Count : 145

All Folders Successfully Dequeued


In [34]:
files_heapq = []

for prefix_deque in folder_prefix_list:
    ooo = s3.list_objects_v2(Bucket=bucket,
                            Prefix=prefix_deque,
                            MaxKeys=max_keys,
                            Delimiter=delimiter)
    for item in ooo['Contents']:
        files = item['Key']
        heapq.heappush(files_heapq, files)

print(f"HeapQueued Files Count : {len(files_heapq)}")

files_sorted = [heappop(files_heapq) for i in range(len(files_heapq))]

Dequeued Files Count : 144484



In [48]:
files_sorted[-5:]

['segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1652313600000/1652401446281.a1d3b8d1ab53.8624813.0982b2b8-ce0d-477a-a66b-be96227fd569.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1661644800000/1661657543271.events-archiver-36.a254827.15747224-726a-4ab3-8cc6-fba8560d3977.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1658534400000/1658554457679.events-archiver-6.a254827.4fbf035f-a300-450c-b8a9-1768ab97f8ac.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1658534400000/1658554466849.events-archiver-23.a254827.b238d945-ac50-4332-bfcc-81f7c4fbc4dd.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1652313600000/1652401474086.9e37f35030d8.8624813.c24db0a9-98c2-4706-a061-893c50cbe8a0.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1664582400000/1664603699843.events-archiver-57.16d05ba.85602dab-eb99-4558-bcc3-243e4d5173d2.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1661644800000/1661657674557.events-archiver-23.a254827.7e7d53a9-8e14-4275-83c6-bbd60e5b60d2.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1663200000000/1663209256663.ev